# Packages

In [1]:
using Gen
using Distributions

# Model Specification

$$Y \text{~} N(\alpha + \beta X, \sigma)$$

where:

* $\alpha$ follows $N(0, 1)$
* $\beta$ follows $N(5, 5)$
* $\sigma$ follows $exp(1)$

# Parameter Values

In [2]:
y = 10.0
x = 3
α = 0.2
β = 7.0
σ = 1
;

# Manual Calculation

In [3]:
l_parameters = Distributions.logpdf(Normal(0, 1), α) +
    Distributions.logpdf(Normal(5, 5), β) +
    Distributions.logpdf(Exponential(1), σ)

ll_data = Distributions.logpdf(Normal(α + β * x, σ), y)

log_probability_manual = l_parameters + ll_data

@show l_parameters
@show ll_data
@show log_probability_manual
;

l_parameters = -4.5473149788434455
ll_data = -63.638938533204666
log_probability_manual = -68.18625351204811


# Using Gen

In [4]:
@gen function model(x)
    α = @trace(normal(0, 1), :α)
    β = @trace(normal(5, 5), :β)
    σ = @trace(exponential(1), :σ)
    
    μ = α + β * x
    
    @trace(normal(μ, σ), :y)
end
;

In [5]:
choices = choicemap(:α => α,
                    :β => β,
                    :σ => σ,
                    :y => y)
;

In [6]:
trace, _ = generate(model, (x, ), choices);

In [7]:
log_probability_gen = get_score(trace)

-68.18625351204811

In [8]:
@assert(log_probability_manual == log_probability_gen)

Check individual parameter score:

In [9]:
for (k, v) in trace.trie.leaf_nodes
    @show k, v.score
end

(k, v.score) = (:α, -0.9389385332046727)
(k, v.score) = (:σ, -1.0)
(k, v.score) = (:y, -63.638938533204666)
(k, v.score) = (:β, -2.6083764456387732)


In [10]:
project(trace, select(:α, :β, :y)) 

-67.18625351204811

In [11]:
[-0.9389385332046727  -2.6083764456387732 -63.638938533204666] |> sum

-67.18625351204811